# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the dataset `retail_sales` from Ironhack's database. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per store.
    - A table for the aggregate per item.

## Instructions
* Read the csv you can find in Ironhack's database.
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [1]:
# your code here

import pandas as pd
import pymysql
from sqlalchemy import create_engine

driver   = 'mysql+pymysql:'
user     = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
ip       = '34.65.10.136'
database = 'retail_sales'
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [2]:
pd.read_sql('SHOW TABLES;', engine)

,Tables_in_retail_sales
0,raw_sales


In [3]:
raw_sales_df = pd.read_sql('SELECT * FROM raw_sales', engine)
raw_sales_df.tail()

,date,shop_id,item_id,item_price,item_cnt_day
4540,2015-01-04,15,4240,1299.0,1.0
4541,2015-01-04,14,21922,99.0,1.0
4542,2015-01-04,15,1969,3999.0,1.0
4543,2015-01-04,14,22091,179.0,1.0
4544,2015-01-04,15,1007,1199.0,1.0


In [12]:
# Checking what the df looks like
raw_sales_df.describe()

,shop_id,item_id,item_price,item_cnt_day
count,4545.000000,4545.000000,4545.000000,4545.000000
mean,34.021122,11140.459406,1031.686121,1.103630
std,16.565517,6558.649572,2073.919990,0.536967
min,2.000000,30.000000,3.000000,-1.000000
25%,22.000000,4977.000000,249.000000,1.000000
50%,31.000000,11247.000000,479.000000,1.000000
75%,50.000000,16671.000000,1192.000000,1.000000
max,59.000000,22162.000000,27990.000000,10.000000


In [5]:
# checking for repetitive in columns
for col in raw_sales_df.columns:
    print(col)
    uni = raw_sales_df[col].unique()
    if len(uni)<20:
        print('\t',uni)

date
	 ['2015-01-04T00:00:00.000000000']
shop_id
item_id
item_price
item_cnt_day
	 [ 1.  2.  6.  3. -1.  4.  5. 10.]


In [6]:
# checking for types
raw_sales_df.dtypes

date            datetime64[ns]
shop_id                  int64
item_id                  int64
item_price             float64
item_cnt_day           float64
dtype: object

In [7]:
# Checking for empty values
raw_sales_df.isna().sum()

date            0
shop_id         0
item_id         0
item_price      0
item_cnt_day    0
dtype: int64

In [21]:
# Labelling the prices for outliers
labels_list = ['Cheap', 'Moderate', 'Expensive', 'Very Expensive', 'Luxury']
bins = pd.cut(raw_sales_df['item_price'], 
                  len(labels_list), 
                  labels=labels_list)
bins_series = pd.Series(bins)
bins_series
raw_sales_df['price_range'] = bins_series
raw_sales_df

,date,shop_id,item_id,item_price,item_cnt_day,price_range
0,2015-01-04,29,1469,1199.0,1.0,Cheap
1,2015-01-04,28,21364,479.0,1.0,Cheap
2,2015-01-04,28,21365,999.0,2.0,Cheap
3,2015-01-04,28,22104,249.0,2.0,Cheap
4,2015-01-04,28,22091,179.0,1.0,Cheap
...,...,...,...,...,...,...
4540,2015-01-04,15,4240,1299.0,1.0,Cheap
4541,2015-01-04,14,21922,99.0,1.0,Cheap
4542,2015-01-04,15,1969,3999.0,1.0,Cheap
4543,2015-01-04,14,22091,179.0,1.0,Cheap


In [26]:
# Checking if there are any bad labeled prices
raw_sales_df[raw_sales_df.item_price < 0]

,date,shop_id,item_id,item_price,item_cnt_day,price_range


In [27]:
# Checking if there are any items returns
raw_sales_df[raw_sales_df.item_cnt_day < 0]

,date,shop_id,item_id,item_price,item_cnt_day,price_range
179,2015-01-04,35,7877,3990.00,-1.0,Cheap
305,2015-01-04,25,2575,2099.00,-1.0,Cheap
386,2015-01-04,21,2946,449.00,-1.0,Cheap
391,2015-01-04,21,1523,799.00,-1.0,Cheap
825,2015-01-04,52,16677,332.67,-1.0,Cheap
899,2015-01-04,44,14652,199.00,-1.0,Cheap
901,2015-01-04,44,8095,499.00,-1.0,Cheap
926,2015-01-04,44,1114,299.00,-1.0,Cheap
1161,2015-01-04,42,1878,2599.00,-1.0,Cheap
1416,2015-01-04,19,2690,1598.00,-1.0,Cheap


In [28]:
# Checking if there are any item_price outliers
pd.Series(raw_sales_df.item_price.unique()).value_counts(bins=5)

(-24.988, 5600.4]     213
(22392.6, 27990.0]      4
(5600.4, 11197.8]       4
(16795.2, 22392.6]      1
(11197.8, 16795.2]      1
dtype: int64

## Functions

### Funtion to add a column by multiplying two values

In [91]:
# Function to create a new column with item revenue per sell
def column_multiply(table_price, table_cnt):
    items_sold = table_price * table_cnt
    return items_sold


### Function to find the top 5

In [95]:
# aggregate sold by item
def top5(df,group_series,agg_series):
    sum_shop = df.groupby(group_series).agg({agg_series:'sum'})
    top5_result = sum_shop.sort_values(agg_series, ascending=False).head(5)
    return top5_result

# Final Function

In [96]:
# adding a new column with item_revenue
raw_sales_df['item_revenue'] = column_multiply(raw_sales_df['item_price'],raw_sales_df['item_cnt_day'])

print('\n The top 5 best selling shops are:\n', top5(raw_sales_df, 'shop_id', 'item_revenue'))

print('\n The top 5 best selling items are:\n',top5(raw_sales_df,'item_id', 'item_cnt_day'))


 The top 5 best selling shops are:
          item_revenue
shop_id              
42           330111.0
31           304692.0
12           295173.0
25           288432.0
21           228999.0

 The top 5 best selling items are:
          item_cnt_day
item_id              
20949            93.0
1969             66.0
21364            66.0
17717            60.0
11927            51.0
